## One-unit neural network model for logistic regression

In this notebook, we illustrate how to do [logistic regression](https://en.wikipedia.org/wiki/Logistic_regression)
using a neural network-like implementation with one single unit (perceptron).
This is equivalent to a logistic regression model, only it is solved using neural networks (forward and backpropagation) rather than with maximum likelihood (or similar algorithms).

As example data, we use the famous [Iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set) 

<img src="logistic_perceptron.png" alt="perceptron" style="width: 500px;"/>

In [1]:
import numpy as np
import tensorflow as tf

## setting seeds
tf.keras.utils.set_random_seed(10)
tf.config.experimental.enable_op_determinism()

ModuleNotFoundError: No module named 'tensorflow'